# Import libraries

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import time
import pandas as pd
#import torch

from huggingface_hub import login
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from peft import PeftModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


# Set Paths and Hyperparameters

In [2]:
# Base path
base_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..', '..'))

# Best model stored under
best_model = "epoch_3"

# Source and target language
source_language = "Early Modern Bohemian German"
target_language = "English"

# Translation direction
translation_direction = "DE_to_EN" if source_language == "Early Modern Bohemian German" else "EN_to_DE"

# Model parameters
unsloth_model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit'
company_name = 'meta'

model_name = unsloth_model_name.split('/')[1]
max_new_tokens = 5000           # Maximum number of model output
max_seq_length = 128000         # Maximum of input tokens
dtype = None                    # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True             # Use 4bit quantization to reduce memory usage. Can be False.

# Load lora model from path
load_lora_model_path = os.path.join(
    base_path, 
    'models', 
    company_name, 
    model_name, 
    'finetuning', 
    translation_direction, 
    best_model)

# Save inference dataset
save_path = os.path.join(
    base_path, 
    'results', 
    company_name,
    model_name, 
    'finetuning', 
    translation_direction, 
    f'{translation_direction}_finetuning.json'
)

# Print paths
print(f'Company name: {company_name}')
print(f'Model name: {model_name}')
print(f'Base path: {base_path}')
print(f'Translation direction: {translation_direction}')
print(f'LoRA model path: {load_lora_model_path}')
print(f'Save finetuning inference dataset path: {save_path}')

# Hugging face login
hub_token = "hf_..."
login(hub_token, add_to_git_credential=True)

Company name: meta
Model name: Meta-Llama-3.1-8B-Instruct-bnb-4bit
Base path: /cs/student/msc/csml/2023/ngriessh/historical_mt
Translation direction: DE_to_EN
LoRA model path: /cs/student/msc/csml/2023/ngriessh/historical_mt/models/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit/finetuning/DE_to_EN/epoch_3
Save finetuning inference dataset path: /cs/student/msc/csml/2023/ngriessh/historical_mt/results/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit/finetuning/DE_to_EN/DE_to_EN_finetuning.json
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /cs/student/msc/csml/2023/ngriessh/.cache/huggingface/token
Login successful


# Load Dataset & ICL examples

In [3]:
# Load test dataset
dataset = load_dataset('niclasgriesshaber/EarlyModernGerman_to_EN_finetuning')
test_dataset = dataset['test']
print(f'Loaded test dataset: \n {test_dataset}')

Loaded test dataset: 
 Dataset({
    features: ['Early Modern Bohemian German', 'English'],
    num_rows: 1000
})


# Prompt template

In [4]:
# Prompt template
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant tasked with translating from {} to {}. NEVER provide an introduction to the translation (e.g. 'Here is the translation:', 'Translate to', 'Hier ist die Übersetzung:', etc.), explanations or clarifications.
NEVER provide a note after your translation.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

### Translation Task. Only translate the following text. Nothing else!

{}:
{}

Translate to {} and match the structure of the source text. Output only this translation and nothing else.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>{}"""

In [5]:
# Apply prompt template to test dataset
def formatting_prompts_func(examples, source_language, target_language):
    
    source_texts = examples[source_language]
    texts = []

    for source_text in source_texts:
        # Format the prompt with dynamic source and target languages
        text = prompt_template.format(
            source_language,  # Dynamic source language
            target_language,  # Dynamic target language
            source_language,  # Target language for translation request
            source_text,      # Actual source text to translate
            target_language,  # Target language for translation output (no actual output included)
            ""                # Placeholder for the output, left empty for inference
        )
        texts.append(text)

    return {"text": texts}

# Apply Prompt Template to Test Dataset

In [6]:
# Apply prompt template to all test samples
test_dataset = test_dataset.map(
    lambda examples: formatting_prompts_func(examples, source_language, target_language),
    batched=True
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
# Output a text file to check prompt
with open('inference_prompt_check', "w") as f:
    f.write(test_dataset['text'][0])

# Load Model

In [8]:
# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=unsloth_model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090 Ti. Max memory: 23.575 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


# Load LoRA adapters

In [9]:
# Load LoRA adapters from Peft
print('Loading LoRA adapaters')
model = PeftModel.from_pretrained(model, load_lora_model_path)
print('LoRA adapters loaded successfully')
print(f'Lora model path under: {load_lora_model_path}')

Loading LoRA adapaters
LoRA adapters loaded successfully
Lora model path under: /cs/student/msc/csml/2023/ngriessh/historical_mt/models/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit/finetuning/DE_to_EN/epoch_3


# Inference

In [10]:
# Set model to inference mode
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [11]:
# Save inferences in new dataset
def process_dataset(test_dataset, model, tokenizer, max_new_tokens, translation_direction):

    # Extract target language
    if translation_direction == "DE_to_EN":
        target_language = "English"
    else:
        target_language = "Early Modern Bohemian German"
    
    # Convert Hugging Face dataset to Pandas DataFrame
    df = pd.DataFrame(test_dataset)
    
    total_time = 0  # Initialize the total time accumulator

    # Loop through each row in the dataframe
    for i, row in df.iterrows():
        start_time = time.time()  # Start timer for the current inference

        try:
            print(f"Processing test point {i + 1} of {len(df)}")

            # Get the text for the current row
            inputs = tokenizer([row['text']], return_tensors="pt").to("cuda")

            # Generate the model outputs
            outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)

            # Decode the outputs, converting from token IDs back to text
            decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=False)

            # Since decoded_output has only one entry, extract the single output
            output = decoded_outputs[0]

            # Define the exact substring to search for
            search_string = f"<|start_header_id|>assistant<|end_header_id|>"

            # Find the index of the search_string in the output
            start_idx = output.find(search_string)

            # Extract everything after the search_string if it's found
            if start_idx != -1:
                extracted_text = output[start_idx + len(search_string):]
            else:
                extracted_text = 'NA'

            # Manually remove the special token '<|eot_id|>' from the beginning and the end, if present
            extracted_text = extracted_text.replace("<|start_header_id|>assistant<|end_header_id|>", "").strip()
            extracted_text = extracted_text.replace("<|eot_id|>", "").strip()

            # Print the output and extracted text
            print('_________________________________________________')
            print(test_dataset[target_language][i])
            print('_________________________________________________')
            print(extracted_text)
            print('_________________________________________________')

            # Add extracted_text directly to the dataframe at index i
            df.at[i, f'{translation_direction}_finetuning'] = extracted_text

        except Exception as e:
            print(f"Error occurred at test point {i + 1}: {str(e)}")
        
        end_time = time.time()  # End timer for the current inference
        elapsed_time = end_time - start_time  # Calculate elapsed time
        total_time += elapsed_time  # Accumulate total time

        print(f"Time for test point {i + 1}: {elapsed_time:.2f} seconds, Total time: {total_time:.2f} seconds")

    return df

In [12]:
# Call the function
processed_dataset = process_dataset(test_dataset, model, tokenizer, max_new_tokens, translation_direction)

# Save the dataset as a JSON file
processed_dataset.to_json(save_path, orient='records', lines=True, force_ascii=False)
print(f"Dataset saved successfully to {save_path}")

Processing test point 1 of 1000
_________________________________________________
Since they did not split the rafting-wood, they shall for this reason split it within 14 days of today and without any delay on pain of 1 Weiss Schocks, signed 3 Oct 1588
_________________________________________________
Since the rafting-wood is not split, for this reason it shall be split within 14 days of today finally and without delay on pain of 1 Weiss Schocks, signed 3 Oct 1588
_________________________________________________
Time for test point 1: 1.16 seconds, Total time: 1.16 seconds
Processing test point 2 of 1000
_________________________________________________
Since Christoff Brohtsack's daughter in Luesdorf was made pregnant by Hans Apelt in Weisbach with whom she was serving, and for this reason was taken into imprisonment, but because she was supposed to name pledges that she would report back upon earliest summoning, but no-one wanted to enter into pledgeship for her, however the father